# Chicago Crime dataset 2001 - Present day

[Dataset (Export)](https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2/data) Intialising dataset download takes a while.
 https://chicago.suntimes.com/2021/11/26/22639255/dead-end-drug-arrests-drugs-possession-chicago
 

Dataset contains roughly 7,400,000 rows after cleaning.

I recommend converting the file to ***PARQUET*** as it drastically reduces its size and increases speed of commands.

# *USE* [Dashboard](https://blog.streamlit.io/crafting-a-dashboard-app-in-python-using-streamlit/)



## Main Questions:

1. ~~Hows does the median household income in communities effect crime rates?~~ (Change) 
2. How has crime (Types of crime) over the last 20 years changed? (Bar Graph, Cache Top 10 - Each year?)
   - [Shootings](https://abc7chicago.com/chicago-shootings-gun-violence-police-department-february-28/14481046/)
   - 2000 per 100,000 @ a rate of 642 per day average[Incidents](https://bestdiplomats.org/most-dangerous-cities-in-the-us/#:~:text=1%2D%20Detroit%2C%20Michigan&text=In%202018%2C%20Detroit%20reported%20a,261%20homicides%20reported%20in%202018.)
   - Some more modern reports indicate up to 4000 crimes per 100,000. However finding the collaborating data seems hard due to all the different sources. 
   - 
3. ~~How does ethnicity diversity in communities effect crime rates?~~ (Change) - Relevant due to old data?
4. How is crime distributed across the city? (Done) - Map - Highest crime rates in the city. Chance over time?
    - Map, using bounding areas, using crimes per year per district ranking Number 1 + Most crimes that year.
5. Shootings by districs and ethnicist, over time.
   - Shootings by district, shootings by district per year. Most effected races, ages, sexes. 
   - Deadliest time of the day.
   - Days without a shooting per year.
   - Deadliest day.
   - Number of days without a shooting.
   - Most common location.
   - 
6. How did the Covid lock down affect crime during its 1 year effect? (Partially Done) - Line Graph, and bar graph, zoom in on the first week of lockdown.
   - [Holy Moly Article](https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.holycross.edu/media/228226/download%3Fattachment&ved=2ahUKEwjFksLkms6FAxXhyAIHHdfFAYUQFnoECBAQAQ&usg=AOvVaw0AmLWmE-kw0kJqDJK-MdGb)
   - Did shootings increase, or decrease? There was still on everyday but less?
   - 40 murders per 100,000. 19th highest in the world, second highest in the USA.
   - Discounting a single country with 5.5 murders per 100,000. The highest in Europe is 2.7. Drastically lower than Chicago.  
7. Graph on drugs over time. Include research. Use the longterm graph and research to justify the drop in drug busts. Ordering via phone, online, "No Go" zones.
   - High rates of "legal" prescribtions, opiodid epidemic. Lower in recent times. 
   - Increase in overdoses due to fentanyl.
   - [Article One](https://www.chicagoappleseed.org/2022/06/15/dynamics-of-drug-possession-charges-in-illinois/)
   - [Article Two](https://chicago.suntimes.com/2021/11/26/22639255/dead-end-drug-arrests-drugs-possession-chicago)
   - [Article Three](https://chicago.suntimes.com/2022/11/10/23444935/drug-possession-jail-safe-t-act-pretrial-fairness-watchdogs-law-enforcement-cook-county-editorial)
   - [Article Four](https://news.wttw.com/2021/12/07/sun-times-bga-report-reveals-costly-toll-dead-end-drug-arrests)


# Notes for Kristian R.
1. Points that worked well so far?

   - Overall data exploration has been good, all algorithms where made on a reduced dataframe of 500k to increase speed of development.

2. What are the main challenges so far?

   - Working with multiple dataframes posing a few issues.
   - Maps, and displaying the correct information.
   - Am I focusing on ethnicity or median income to much, it tells a story but to which end? Data can be easily manipulated. 

3. Preliminary findings.

   - Firstly, becareful where you go in Chicago.
   - Crime rates are highly effected by median income and ethnicity.
   - Crime rates have been falling over the last 20 years, but unlike most major citys population has barely changed.
   - The types of crimes have changed.
   - Narcotics crimes have fallen drastically due to lack in procutions, in turn lack of arrests as waste of time. 72% of cases thrown out. 93% of all arrests under 1g are      never recorded.
   - Austin - Community #25 is disproportionately responsible for the majority of crime across the city.
   - Covid had a major effect on crime, but rebounded quickly.
  


# Notes today:

- Summarise the majority of the data.

In [7]:
import folium
from folium import plugins
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import geopandas as gpd

df = pd.read_parquet("data/crime_data.parquet")

In [8]:
#Data Cleaning
columns_to_delete = ['Case Number', 'Block', 'Ward', 'FBI Code', 'X Coordinate', 'Y Coordinate', 'Updated On', 'Location', 'Beat', 'Description','Location Description'] 
df = df.drop(columns=columns_to_delete)
df = df.dropna(subset=['District', 'Community Area','Latitude', 'Longitude' ])
df = df.astype({'District':'int', 'Community Area':'int'})
filtered_df = df

In [11]:
# Filter data for the years 2002 to 2023
filtered_data_top_15_crimes = filtered_df[(filtered_df['Year'] >= 2002) & (filtered_df['Year'] <= 2023)]

# Group by 'Year' and 'Primary Type', then count occurrences
crime_counts_per_year = filtered_data_top_15_crimes.groupby(['Year', 'Primary Type']).size().reset_index(name='Count')

# Sort the data within each year to find the top 10 crimes
top_15_crimes_per_year = crime_counts_per_year.groupby('Year').apply(lambda x: x.nlargest(15, 'Count')).reset_index(drop=True)
filtered_data_top_15_crimes['Primary Type'].value_counts()

# Display the top 10 crimes per year
#top_15_crimes_per_year.to_parquet('data/data_cache/top_10_crimes_per_year.parquet', index=False)


/var/folders/h0/8zhlmx892psg97sq1cyqlzx00000gn/T/ipykernel_76908/2213638584.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_15_crimes_per_year = crime_counts_per_year.groupby('Year').apply(lambda x: x.nlargest(15, 'Count')).reset_index(drop=True)


Primary Type
THEFT                         1541462
BATTERY                       1331806
CRIMINAL DAMAGE                833664
NARCOTICS                      672648
ASSAULT                        482167
OTHER OFFENSE                  451181
BURGLARY                       395591
MOTOR VEHICLE THEFT            359390
DECEPTIVE PRACTICE             320819
ROBBERY                        276303
CRIMINAL TRESPASS              199348
WEAPONS VIOLATION              106516
PROSTITUTION                    61458
OFFENSE INVOLVING CHILDREN      50415
PUBLIC PEACE VIOLATION          49327
Name: count, dtype: int64

In [5]:
filtered_df['Date'] = pd.to_datetime(filtered_df['Date'])

# Filter data for the dates from March 2019 to March 2023
filtered_df_crimes_commited = filtered_df[(filtered_df['Date'] >= '2019-03-01') & (filtered_df['Date'] < '2023-04-01')]

# Group by month and count the crimes
crime_counts_per_month = filtered_df_crimes_commited.groupby(pd.Grouper(key='Date', freq='M')).size().reset_index(name='Count')

crime_counts_per_month

/var/folders/h0/8zhlmx892psg97sq1cyqlzx00000gn/T/ipykernel_76034/3911023096.py:7: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  crime_counts_per_month = filtered_df_crimes_commited.groupby(pd.Grouper(key='Date', freq='M')).size().reset_index(name='Count')


,Date,Count
0,2019-03-31,20324
1,2019-04-30,20912
2,2019-05-31,23536
3,2019-06-30,23475
4,2019-07-31,24702
5,2019-08-31,24246
6,2019-09-30,22284
7,2019-10-31,21541
8,2019-11-30,19743
9,2019-12-31,20567


In [6]:
# Aggregate the total number of crimes by district
crimes_by_district = filtered_df.groupby('Community Area').size().reset_index(name='Total Crimes')

crimes_by_district.to_parquet('data/data_cache/crimes_by_district.parquet', index=False)

In [7]:
geo_boundary_chicago = gpd.read_file('data/chicago_boundaries.geojson')

#Data handling
#total_crime_count = filtered_df.groupby('Community Area').size().reset_index(name='Crimes')
geo_boundary_chicago['community'] = geo_boundary_chicago['area_num_1'].astype(int)
crimes_by_district['Community Area'] = crimes_by_district['Community Area'].astype(int)
geo_boundary_chicago_merged = geo_boundary_chicago.merge(crimes_by_district, left_on='community', right_on='Community Area', how='left')


geo_boundary_chicago_merged


,community,area,shape_area,perimeter,area_num_1,area_numbe,comarea_id,comarea,shape_len,geometry,Community Area,Total Crimes
0,35,0,46004621.1581,0,35,35,0,0,31027.0545098,"MULTIPOLYGON (((-87.60914 41.84469, -87.60915 ...",35,81706
1,36,0,16913961.0408,0,36,36,0,0,19565.5061533,"MULTIPOLYGON (((-87.59215 41.81693, -87.59231 ...",36,17112
2,37,0,19916704.8692,0,37,37,0,0,25339.0897503,"MULTIPOLYGON (((-87.62880 41.80189, -87.62879 ...",37,24227
3,38,0,48492503.1554,0,38,38,0,0,28196.8371573,"MULTIPOLYGON (((-87.60671 41.81681, -87.60670 ...",38,101585
4,39,0,29071741.9283,0,39,39,0,0,23325.1679062,"MULTIPOLYGON (((-87.59215 41.81693, -87.59215 ...",39,42799
...,...,...,...,...,...,...,...,...,...,...,...,...
72,74,0,75584290.0209,0,74,74,0,0,48665.1305392,"MULTIPOLYGON (((-87.69646 41.70714, -87.69644 ...",74,16404
73,75,0,91877340.6988,0,75,75,0,0,46396.419362,"MULTIPOLYGON (((-87.64215 41.68508, -87.64249 ...",75,58289
74,76,0,371835607.687,0,76,76,0,0,173625.98466,"MULTIPOLYGON (((-87.83658 41.98640, -87.83658 ...",76,43787
75,77,0,48449990.8397,0,77,77,0,0,31004.8309456,"MULTIPOLYGON (((-87.65456 41.99817, -87.65456 ...",77,74006


In [8]:
gun_crime_data = pd.read_csv('data/gun_crime_chicago.csv')

shootings_and_deaths = gun_crime_data.groupby('COMMUNITY_AREA').agg(
    Homicides=('VICTIMIZATION_PRIMARY', lambda x: (x == 'HOMICIDE').sum()),
    Shootings=('GUNSHOT_INJURY_I', lambda x: (x == 'YES').sum())
).reset_index()
shootings_and_deaths.head(25)
shootings_and_deaths.to_parquet('data/data_cache/shootings_and_deaths.parquet', index=False)

In [9]:

gun_crime_data['DATE'] = pd.to_datetime(gun_crime_data['DATE'])

# Filter data to include only records from 2010 onwards
gun_crime_data = gun_crime_data[gun_crime_data['DATE'].dt.year >= 2010]

# Group data by community area, race, exact age, and sex, then count occurrences
victim_counts = gun_crime_data.groupby(['COMMUNITY_AREA', 'RACE', 'AGE', 'SEX']).size().reset_index(name='Count')

# Display the first few rows of the aggregated data
print(victim_counts.head())


/var/folders/h0/8zhlmx892psg97sq1cyqlzx00000gn/T/ipykernel_76034/2704657215.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  gun_crime_data['DATE'] = pd.to_datetime(gun_crime_data['DATE'])


  COMMUNITY_AREA RACE    AGE SEX  Count
0    ALBANY PARK  API   0-19   F      1
1    ALBANY PARK  API   0-19   M      1
2    ALBANY PARK  API  20-29   F      1
3    ALBANY PARK  API  20-29   M      3
4    ALBANY PARK  API  30-39   M      1


In [15]:
filtered_df = filtered_df[filtered_df['Year'] >= 2002]

# Group by 'Year' and 'Community Area' and count the number of crimes
crime_counts_per_year = filtered_df.groupby(['Year', 'Community Area']).size().reset_index(name='Total Crimes')

# Pivot the table to have years as columns and community areas as rows
crime_counts_pivot = crime_counts_per_year.pivot(index='Community Area', columns='Year', values='Total Crimes').fillna(0)

# Reset index to make 'Community Area' a column again
crime_counts_pivot.reset_index(inplace=True)

# Display the first few rows of the pivoted DataFrame
print(crime_counts_pivot.head())
crime_counts_pivot.to_parquet('data/data_cache/crime_counts_every_year_community.parquet', index=False)

Year  Community Area    2002    2003    2004    2005    2006    2007    2008  \
0                  0     6.0     7.0    15.0     2.0    12.0     6.0     7.0   
1                  1  5899.0  7628.0  7587.0  7484.0  7155.0  6784.0  6386.0   
2                  2  3940.0  5195.0  5735.0  5807.0  6024.0  5250.0  5376.0   
3                  3  5864.0  7464.0  7413.0  7490.0  6780.0  6382.0  5902.0   
4                  4  2573.0  3251.0  3342.0  3350.0  3268.0  2840.0  2743.0   

Year    2009    2010  ...    2015    2016    2017    2018    2019    2020  \
0        5.0     3.0  ...     0.0     0.0     0.0     0.0     0.0     0.0   
1     5876.0  5568.0  ...  3509.0  3646.0  4085.0  3753.0  4007.0  3204.0   
2     5163.0  4880.0  ...  3054.0  3269.0  3548.0  3484.0  3441.0  3050.0   
3     5125.0  5069.0  ...  3569.0  3542.0  3509.0  3578.0  3313.0  2867.0   
4     2606.0  2559.0  ...  1740.0  1974.0  1965.0  1914.0  1778.0  1826.0   

Year    2021    2022    2023   2024  
0        0.0     0

In [23]:
crimes_count = filtered_df['Primary Type'].value_counts()
crimes_count
crimes_count.to_csv('data/data_cache/crime_count_total_amount.csv')


In [5]:

df_2 = pd.read_parquet("data/data_cache/1_top_10_crimes_per_year.parquet")
df_2.to_csv('data/temp/top_10.csv')

In [15]:
filtered_df['Hour'] = filtered_df['Date'].dt.hour

# Calculate the average number of crimes per hour
average_crimes = filtered_df.groupby('Hour').size()
average_crimes_per_hour = average_crimes.reset_index(name='Occurrences')
average_days = filtered_df['Date'].dt.date.nunique()
average_crimes_per_hour['Average'] = average_crimes_per_hour['Occurrences'] / average_days

# Ensure all hours are represented in the data
all_hours = pd.DataFrame({'Hour': range(24)})
average_crimes_per_hour = pd.merge(all_hours, average_crimes_per_hour[['Hour', 'Average']], on='Hour', how='left').fillna(0)
average_crimes_per_hour

,Hour,Average
0,0,865.224953
1,1,0.000000
2,2,0.000000
3,3,0.000000
4,4,0.000000
5,5,0.000000
6,6,0.000000
7,7,0.000000
8,8,0.000000
9,9,0.000000


In [16]:
df_2 = pd.read_parquet("data/data_cache/2a_geo_boundary_chicago_merged.parquet")
df_2.to_csv('data/temp/geo.csv', index=False)